In [1]:
import pandas as pd
import numpy as np
import re
import json
from datetime import timedelta, datetime

In [2]:
import os,sys,inspect
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
sys.path.insert(0,parentdir) 
from statistic import Statistic
from converter import Converter
pd.set_option('display.max_colwidth', -1)

In [3]:
# for autoreload modules
%load_ext autoreload
%autoreload 2

In [8]:
path_data = '/mnt/files/workdata/work/merchants/merchant_26_tutu2/01_raw_data/data-rail-with-id_2020-02-04/'
             
#input_file = path_data + 'data-rail-with-id.csv'
input_file = path_data + '40000_data-rail-with-id.csv'

In [5]:
output_file = path_data + 'tmp-correct-data_with-id.csv'

In [10]:
df = pd.read_csv(input_file, sep='\t', dtype=str, keep_default_na=False)#, nrows=100
df.shape

(39999, 5)

In [11]:
col_names = Converter.merge_flat_dictionary_keys_from_json_column(df, 'json', is_print_error=0)

In [12]:
warning_data = Converter.write_json_column_to_csv_read_data_from_file(input_file, col_names, output_file, is_print_error = 2)

In [13]:
df_warning  = pd.DataFrame.from_dict(warning_data)        

In [14]:
mask = df_warning.problem_lines.str.contains('NULL' )

In [17]:
df_warn_json = df_warning[~mask]

In [18]:
df_warn_json.iloc[:, :]

,problem_lines,problem,problem_nums
1,"{""train"":{""is_trailer"":false,""is_firm"":false,""departure_date"":""2019-08-01 16:02:00"",""departure"":""Санкт-Петербург Ладож."",""arrival"":""Челябинск""},""client"":{""name"":""Иван Скворов"",""account_exists"":false,""email"":""aaaaaaaa_aaaaaaa_2018@mail.ru\\n"",""phone"":""7912701****"",""account_confirmed"":false},""order"":{""has_rzd_bonus"":false,""is_composite_trip"":false,""seats_selected"":true,""site_zone"":""main"",""requested_fin_documents"":false,""sms_notification"":true,""is_round_trip"":false},""passengers"":[{""doc_country"":""RUS"",""birthday "":""1998-06-07"",""seat_side"":false,""seat_level"":""bottom"",""seat"":""23"",""doc_type"":""inner"",""has_middle_name "":true,""sex"":""M""}],""amount"":""4583.00"",""id"":""25263290"",""date"":""2019-05-22 10:44:29"",""location"":{""ip"":""217.66.157.85""},""device"":{""type"":""Redmi 4X"",""http_headers"":{""user_agent"":""android Redmi 4X 25 REL""},""browser_version"":""3.2.0"",""browser"":""tuturu""},""currency"":""RUB"",""trip"":{""passengers"":1,""registration_available"":true,""departure_date"":""2019-08-01 16:02:00"",""adults"":1,""departure_country"":""RUS"",""departure_longitude"":""30.441575000"",""arrival_country"":""RUS"",""include_laundry"":true,""children"":0,""infants"":0,""arrival_longitude"":""49.651568849"",""has_laundry_option"":false,""arrival"":""Киров Пасс"",""departure_latitude"":""59.931762600"",""car_category"":""4"",""arrival_latitude"":""58.578579739"",""insurance_selected"":false,""departure"":""Санкт-Петербург Ладож."",""travel_duration"":22,""arrival_date"":""2019-08-02 14:11:00"",""days_to_departure"":71}}",aaaaaaaa_aaaaaaa_2018@mail.ru\\n,15306
107,"{""date"":""2019-07-18 17:48:10"",""location"":{""ip"":""83.171.108.156""},""id"":""26848271"",""amount"":""4022.70"",""trip"":{""insurance_selected"":null,""departure_latitude"":""59.918844200"",""car_category"":""3"",""arrival_latitude"":""52.430868892"",""days_to_departure"":14,""travel_duration"":16,""departure"":""Санкт-Петербург-Витеб."",""arrival_date"":""2019-08-02 08:58:00"",""infants"":0,""children"":0,""include_laundry"":false,""arrival"":""Гомель-Пасс."",""arrival_longitude"":""30.991198327"",""has_laundry_option"":false,""adults"":1,""departure_country"":""RUS"",""arrival_country"":""RUS"",""departure_longitude"":""30.330026700"",""passengers"":1,""registration_available"":true,""departure_date"":""2019-08-01 17:20:00""},""currency"":""RUB"",""device"":{""type"":""PC"",""http_headers"":{""user_agent"":""Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/75.0.3770.142 Safari/537.36""},""browser_version"":""75"",""browser"":""chrome""},""train"":{""arrival"":""Гомель-Пасс."",""departure"":""Санкт-Петербург-Витеб."",""departure_date"":""2019-08-01 17:20:00"",""is_trailer"":false,""is_firm"":false},""passengers"":[{""sex"":""M"",""has_middle_name "":true,""seat_level"":""top"",""seat_side"":false,""seat"":""2"",""doc_type"":""foreign"",""birthday "":""1978-08-14"",""doc_country"":""BLR""}],""order"":{""is_round_trip"":false,""sms_notification"":true,""requested_fin_documents"":false,""site_zone"":""main"",""is_composite_trip"":false,""seats_selected"":true,""has_rzd_bonus"":false},""client"":{""account_exists"":false,""name"":""Парк- Отель Гора\\"""",""email"":""aaaa@medvezhiagora.ru"",""phone"":""7911016****"",""account_confirmed"":false}}",Парк- Отель Гора\\,27663
